# GRU Classification with MPQA Dataset
<hr>

We will build a text classification model using GRU model on the MPQA Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/MPQA/MPQA.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10606, 3)


sentence  label  split
0              complaining      0  train
1       failing to support      0  train
2        desperately needs      0  train
3      many years of decay      0  train
4             no quick fix      0  train
...                    ...    ...    ...
10601                urged      1  train
10602       strictly abide      1  train
10603                 hope      1  train
10604       strictly abide      1  train
10605                           1  train

[10606 rows x 3 columns]

In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10606 entries, 0 to 10605
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10606 non-null  object
 1   label     10606 non-null  int32 
 2   split     10606 non-null  object
dtypes: int32(1), object(2)
memory usage: 207.3+ KB


In [5]:
corpus.groupby( by='label').count()

sentence  split
label                 
0          7294   7294
1          3312   3312

In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'complaining'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  no quick fix
Into a sequence of int: [25, 945, 1476]
Into a padded sequence: [  25  945 1476    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
of 3
to 4
a 5
and 6
not 7
is 8
in 9
be 10
6236


# Model 1: Embedding Random
<hr>

<img src="model.png" style="width:700px;height:400px;"> <br>

## GRU Model

In [11]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [12]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               140544    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 440,673
Trainable params: 440,673
Non-trainable params: 0
_________________________________________________________________


In [27]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [28]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1

    # Define the input shape
    model = define_model(input_dim=vocab_size, max_length=max_len)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=30, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/30
299/299 [==============================] - 81s 224ms/step - loss: 0.5523 - accuracy: 0.7316 - val_loss: 0.3673 - val_accuracy: 0.8549
Epoch 2/30
299/299 [==============================] - 62s 208ms/step - loss: 0.2079 - accuracy: 0.9272 - val_loss: 0.3677 - val_accuracy: 0.8511
Epoch 3/30
299/299 [==============================] - 64s 214ms/step - loss: 0.1302 - accuracy: 0.9556 - val_loss: 0.4196 - val_accuracy: 0.8426
Epoch 4/30
299/299 [==============================] - 65s 219ms/step - loss: 0.0987 - accuracy: 0.9646 - val_loss: 0.4448 - val_accuracy: 0.8492
Epoch 5/30
299/299 [==============================] - 65s 219ms/step - loss: 0.0796 - accuracy: 0.9699 - val_loss: 0.5306 - val_accuracy: 0.8426
Epoch 6/30
299/299 [==============================] - 65s 219ms/step - loss: 0.0608 - accuracy: 0.9746 - val_loss: 0.5837 - val_accuracy: 0.8322
Epoch 7/30
299/299 [==============================] - 65s 217ms/step - loss: 0.0496 - accuracy: 0.9798 - val_loss: 0.

Epoch 9/30
299/299 [==============================] - 69s 232ms/step - loss: 0.0430 - accuracy: 0.9814 - val_loss: 0.6089 - val_accuracy: 0.8464
Epoch 10/30
299/299 [==============================] - 69s 231ms/step - loss: 0.0385 - accuracy: 0.9832 - val_loss: 0.6631 - val_accuracy: 0.8369
Epoch 11/30
299/299 [==============================] - 69s 232ms/step - loss: 0.0336 - accuracy: 0.9863 - val_loss: 0.6521 - val_accuracy: 0.8388
Epoch 12/30
299/299 [==============================] - 69s 231ms/step - loss: 0.0373 - accuracy: 0.9837 - val_loss: 0.6265 - val_accuracy: 0.8501
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 86.42789721488953
Training 6: 
Epoch 1/30
299/299 [==============================] - 88s 255ms/step - loss: 0.5478 - accuracy: 0.7366 - val_loss: 0.3749 - val_accuracy: 0.8539
Epoch 2/30
299/299 [==============================] - 75s 251ms/step - loss: 0.2024 - accuracy: 0.9304 - val_loss: 0.3710 - val_accuracy: 0.85

Restoring model weights from the end of the best epoch.
Epoch 00014: early stopping
Test Accuracy: 85.2830171585083
Training 10: 
Epoch 1/30
299/299 [==============================] - 72s 201ms/step - loss: 0.5508 - accuracy: 0.7293 - val_loss: 0.3428 - val_accuracy: 0.8670
Epoch 2/30
299/299 [==============================] - 50s 167ms/step - loss: 0.2130 - accuracy: 0.9264 - val_loss: 0.3522 - val_accuracy: 0.8717
Epoch 3/30
299/299 [==============================] - 49s 165ms/step - loss: 0.1216 - accuracy: 0.9588 - val_loss: 0.3963 - val_accuracy: 0.8642
Epoch 4/30
299/299 [==============================] - 49s 164ms/step - loss: 0.0999 - accuracy: 0.9628 - val_loss: 0.4265 - val_accuracy: 0.8613
Epoch 5/30
299/299 [==============================] - 49s 162ms/step - loss: 0.0748 - accuracy: 0.9732 - val_loss: 0.4726 - val_accuracy: 0.8509
Epoch 6/30
299/299 [==============================] - 48s 162ms/step - loss: 0.0565 - accuracy: 0.9776 - val_loss: 0.4931 - val_accuracy: 0.8689


## Summary

In [29]:
record

acc1       acc2       acc3       acc4       acc5       acc6  \
0  85.485393  84.825635  85.108387  85.485393  86.427897  85.862392   

        acc7       acc8       acc9      acc10        AVG  
0  88.396227  87.452829  85.283017  87.169814  86.149698

In [30]:
report = record
report = report.to_excel('GRU_MPQA.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [31]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [32]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [33]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [34]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 6083 words present from 6236 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [35]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    
    embed_matrix = np.zeros((vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.word_vec(word)
            
        # initialize the unknown word with standard normal distribution values
        else:
            embed_matrix[idx] = np.random.randn(emb_dim)
            
    return embed_matrix

In [36]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i)
em_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.6320836 , -0.39995871,  0.30284232, ..., -0.28593645,
        -0.02975578, -0.58115922],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## GRU Model

In [37]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [38]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_30 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_30 (Bidirectio (None, 128)               140544    
_________________________________________________________________
dropout_30 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 129       
Total params: 440,673
Trainable params: 140,673
Non-trainable params: 300,000
_________________________________________________________________


## Train and Test the Model

In [39]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [40]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_2(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record2 = record2.append(temp, ignore_index=True)
print()
print(record2)
print()

Training 1: 
Epoch 1/40
299/299 [==============================] - 46s 118ms/step - loss: 0.4690 - accuracy: 0.7794 - val_loss: 0.3482 - val_accuracy: 0.8662
Epoch 2/40
299/299 [==============================] - 31s 103ms/step - loss: 0.3094 - accuracy: 0.8782 - val_loss: 0.4063 - val_accuracy: 0.8699
Epoch 3/40
299/299 [==============================] - 30s 101ms/step - loss: 0.2737 - accuracy: 0.8955 - val_loss: 0.4700 - val_accuracy: 0.8718
Epoch 4/40
299/299 [==============================] - 30s 99ms/step - loss: 0.2565 - accuracy: 0.9019 - val_loss: 0.4764 - val_accuracy: 0.8671
Epoch 5/40
299/299 [==============================] - 30s 99ms/step - loss: 0.2309 - accuracy: 0.9101 - val_loss: 0.5298 - val_accuracy: 0.8671
Epoch 6/40
299/299 [==============================] - 29s 98ms/step - loss: 0.2185 - accuracy: 0.9163 - val_loss: 0.6000 - val_accuracy: 0.8728
Epoch 7/40
299/299 [==============================] - 30s 99ms/step - loss: 0.2093 - accuracy: 0.9168 - val_loss: 0.6157

299/299 [==============================] - 33s 109ms/step - loss: 0.2261 - accuracy: 0.9147 - val_loss: 0.4344 - val_accuracy: 0.8690
Epoch 7/40
299/299 [==============================] - 33s 109ms/step - loss: 0.2034 - accuracy: 0.9227 - val_loss: 0.4543 - val_accuracy: 0.8671
Epoch 8/40
299/299 [==============================] - 33s 109ms/step - loss: 0.1863 - accuracy: 0.9275 - val_loss: 0.4710 - val_accuracy: 0.8662
Epoch 9/40
299/299 [==============================] - 33s 109ms/step - loss: 0.1736 - accuracy: 0.9330 - val_loss: 0.4818 - val_accuracy: 0.8596
Epoch 10/40
299/299 [==============================] - 33s 109ms/step - loss: 0.1613 - accuracy: 0.9383 - val_loss: 0.5146 - val_accuracy: 0.8539
Epoch 11/40
299/299 [==============================] - 33s 110ms/step - loss: 0.1427 - accuracy: 0.9514 - val_loss: 0.5536 - val_accuracy: 0.8558
Epoch 12/40
299/299 [==============================] - 33s 110ms/step - loss: 0.1421 - accuracy: 0.9474 - val_loss: 0.5424 - val_accuracy: 

Epoch 9/40
299/299 [==============================] - 26s 88ms/step - loss: 0.1756 - accuracy: 0.9346 - val_loss: 0.6173 - val_accuracy: 0.8765
Epoch 10/40
299/299 [==============================] - 26s 88ms/step - loss: 0.1566 - accuracy: 0.9430 - val_loss: 0.6106 - val_accuracy: 0.8794
Epoch 11/40
299/299 [==============================] - 26s 88ms/step - loss: 0.1493 - accuracy: 0.9434 - val_loss: 0.6469 - val_accuracy: 0.8737
Epoch 12/40
299/299 [==============================] - 26s 88ms/step - loss: 0.1355 - accuracy: 0.9496 - val_loss: 0.7109 - val_accuracy: 0.8690
Epoch 13/40
299/299 [==============================] - 26s 87ms/step - loss: 0.1263 - accuracy: 0.9525 - val_loss: 0.7669 - val_accuracy: 0.8633
Epoch 14/40
299/299 [==============================] - 26s 89ms/step - loss: 0.1516 - accuracy: 0.9431 - val_loss: 0.7683 - val_accuracy: 0.8756
Epoch 15/40
299/299 [==============================] - 26s 88ms/step - loss: 0.1054 - accuracy: 0.9603 - val_loss: 0.7538 - val_acc

Epoch 16/40
299/299 [==============================] - 48s 162ms/step - loss: 0.1135 - accuracy: 0.9601 - val_loss: 0.5846 - val_accuracy: 0.8170
Epoch 17/40
299/299 [==============================] - 49s 162ms/step - loss: 0.0935 - accuracy: 0.9680 - val_loss: 0.5995 - val_accuracy: 0.8198
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Test Accuracy: 85.94339489936829
Training 10: 
Epoch 1/40
299/299 [==============================] - 63s 175ms/step - loss: 0.4792 - accuracy: 0.7663 - val_loss: 0.3379 - val_accuracy: 0.8566
Epoch 2/40
299/299 [==============================] - 47s 158ms/step - loss: 0.2952 - accuracy: 0.8845 - val_loss: 0.3136 - val_accuracy: 0.8689
Epoch 3/40
299/299 [==============================] - 47s 156ms/step - loss: 0.2719 - accuracy: 0.8956 - val_loss: 0.3180 - val_accuracy: 0.8708
Epoch 4/40
299/299 [==============================] - 46s 155ms/step - loss: 0.2521 - accuracy: 0.9028 - val_loss: 0.3692 - val_accuracy: 0.87

## Summary

In [41]:
record2

acc1       acc2       acc3       acc4       acc5       acc6  \
0  87.747407  80.961359  87.087655  80.678606  86.899149  88.030159   

        acc7       acc8       acc9      acc10        AVG  
0  86.886793  87.924528  85.943395  87.641507  85.980056

In [42]:
report = record2
report = report.to_excel('GRU_MPQA_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## GRU Model

In [43]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [44]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_41 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_41 (Bidirectio (None, 128)               140544    
_________________________________________________________________
dropout_41 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 129       
Total params: 440,673
Trainable params: 440,673
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [45]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [46]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_3(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record3 = record3.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/40
299/299 [==============================] - 86s 240ms/step - loss: 0.4582 - accuracy: 0.7847 - val_loss: 0.3267 - val_accuracy: 0.8464
Epoch 2/40
299/299 [==============================] - 68s 228ms/step - loss: 0.1929 - accuracy: 0.9336 - val_loss: 0.3510 - val_accuracy: 0.8407
Epoch 3/40
299/299 [==============================] - 68s 227ms/step - loss: 0.1359 - accuracy: 0.9528 - val_loss: 0.3999 - val_accuracy: 0.8369
Epoch 4/40
299/299 [==============================] - 68s 227ms/step - loss: 0.0970 - accuracy: 0.9665 - val_loss: 0.4513 - val_accuracy: 0.8247
Epoch 5/40
299/299 [==============================] - 68s 227ms/step - loss: 0.0841 - accuracy: 0.9693 - val_loss: 0.4234 - val_accuracy: 0.8549
Epoch 6/40
299/299 [==============================] - 68s 227ms/step - loss: 0.0727 - accuracy: 0.9728 - val_loss: 0.5291 - val_accuracy: 0.8256
Epoch 7/40
299/299 [==============================] - 68s 227ms/step - loss: 0.0538 - accuracy: 0.9789 - val_loss: 0.

299/299 [==============================] - 62s 207ms/step - loss: 0.0343 - accuracy: 0.9856 - val_loss: 0.9518 - val_accuracy: 0.7945
Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Test Accuracy: 81.80961608886719
Training 5: 
Epoch 1/40
299/299 [==============================] - 92s 251ms/step - loss: 0.4463 - accuracy: 0.7957 - val_loss: 0.4504 - val_accuracy: 0.8040
Epoch 2/40
299/299 [==============================] - 71s 237ms/step - loss: 0.1943 - accuracy: 0.9309 - val_loss: 0.4599 - val_accuracy: 0.8124
Epoch 3/40
299/299 [==============================] - 71s 238ms/step - loss: 0.1217 - accuracy: 0.9594 - val_loss: 0.5178 - val_accuracy: 0.7964
Epoch 4/40
299/299 [==============================] - 71s 239ms/step - loss: 0.1052 - accuracy: 0.9611 - val_loss: 0.5927 - val_accuracy: 0.8040
Epoch 5/40
299/299 [==============================] - 71s 238ms/step - loss: 0.0805 - accuracy: 0.9727 - val_loss: 0.6078 - val_accuracy: 0.8011
Epoch 6/40


Epoch 8/40
299/299 [==============================] - 74s 247ms/step - loss: 0.0490 - accuracy: 0.9813 - val_loss: 1.0755 - val_accuracy: 0.7896
Epoch 9/40
299/299 [==============================] - 74s 247ms/step - loss: 0.0438 - accuracy: 0.9821 - val_loss: 1.1356 - val_accuracy: 0.7981
Epoch 10/40
299/299 [==============================] - 73s 246ms/step - loss: 0.0380 - accuracy: 0.9843 - val_loss: 1.1545 - val_accuracy: 0.7953
Epoch 11/40
299/299 [==============================] - 73s 245ms/step - loss: 0.0394 - accuracy: 0.9856 - val_loss: 1.1035 - val_accuracy: 0.8075
Epoch 12/40
299/299 [==============================] - 73s 245ms/step - loss: 0.0374 - accuracy: 0.9823 - val_loss: 1.0357 - val_accuracy: 0.8085
Epoch 13/40
299/299 [==============================] - 72s 242ms/step - loss: 0.0363 - accuracy: 0.9831 - val_loss: 1.0863 - val_accuracy: 0.8009
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping
Test Accuracy: 80.94339370727539
Training 

## Summary

In [47]:
record3

acc1       acc2       acc3       acc4       acc5       acc6  \
0  85.485393  86.616397  82.092369  81.809616  82.092369  77.191329   

        acc7       acc8      acc9      acc10        AVG  
0  86.132073  80.943394  87.26415  83.490568  83.311766

In [48]:
report = record3
report = report.to_excel('GRU_MPQA_3.xlsx', sheet_name='dynamic')